<a href="https://colab.research.google.com/github/rymuelle/10_0_X_alignment_residual_check/blob/master/simpleCFBMinimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import pandas as pd
from google.colab import files, drive
import os
import math

In [0]:
dtype = torch.float
device = torch.device("cpu")
device = torch.device("cuda:0")

Mount drive and get our game CSV:

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
game_history=pd.read_csv('/content/drive/My Drive/Colab Notebooks/CFB_pytorch_toy_minimizer/game_history.csv')
game_history_2019=game_history[game_history.season.fillna(0).astype(int)==2019]
game_history_2019.head()

,team1,team2,team1Home,score1,score2,ordinal_date,season
98528,St Mary's NS,St Fran Xavier,-1,39,7.0,737288.0,2019.0
98529,Acadia,Mt Allison,-1,13,29.0,737288.0,2019.0
98530,Manitoba,Regina,-1,12,9.0,737292.0,2019.0
98531,Coffeyville CC,Iowa Western CC,-1,14,36.0,737293.0,2019.0
98532,Hutchinson CC,Ellsworth CC,-1,41,16.0,737293.0,2019.0


Now, build list and dict of teams. Dict converts between index and name, list stores team power

In [0]:
team_list=pd.concat([game_history_2019.team1,game_history_2019.team2]).unique()
team_list=np.append(team_list,'home_field')
team_dict={team:count for count,team in enumerate(team_list)}

In [0]:
count=0
lengthX=len(team_list)
game_array=[]
spread_array=[]
for index, row in game_history_2019.iterrows():
  game=np.zeros(lengthX)
  game[team_dict[row.team1]]=1
  game[team_dict[row.team2]]=-1
  game[lengthX-1]=row.team1Home
  spread = row.score1-row.score2
  if not math.isnan(spread): 
    game_array.append(game)
    spread_array.append(row.score1-row.score2)
  else:
    print(spread)
  #if count > 1000: break
  count+=1

Our "network" is litterally one node. Team1 power - Team2 power + homefield advantage = predicted score.

Weights here will reflect team power and home field advantage.

In [7]:
team_power=torch.zeros(len(team_list),1, device=device, dtype=dtype, requires_grad=True)
x=torch.tensor(game_array, device=device, dtype=dtype)
y=torch.tensor(spread_array, device=device, dtype=dtype).unsqueeze(1)
print(x.shape)
print(team_power.shape)
print(y.shape)

torch.Size([5105, 928])
torch.Size([928, 1])
torch.Size([5105, 1])


Now, lets optimize team's power:

In [8]:
learning_rate = 1e-4
nGames=len(x)
for t in range(100000):
  pred_spread=x.mm(team_power)
  loss = (pred_spread - y).pow(2).sum()
  loss.backward()
  with torch.no_grad():
    team_power-=learning_rate*team_power.grad
    team_power.grad.zero_()
  loss=math.sqrt(loss.item()/(nGames))
  if t%10000==0:print(t,loss)

0 25.353848421388236
10000 13.22135900678474
20000 13.081134313450903
30000 13.014639914564622
40000 12.973865276604155
50000 12.946491059517346
60000 12.927355797447905
70000 12.913694907369727
80000 12.90382661964933
90000 12.896645673880094


Finally, we can print out our ranking:

In [9]:
team_rank = []
for index, power in enumerate(team_power):
  if power !=0 and team_list[index]!='home_field':
      team_rank.append([team_list[index],float(power)])
  elif power !=0: print(team_list[index],float(power))

team_rank=sorted(team_rank, key=lambda team_rank:team_rank[1],reverse=True)
for index, team in enumerate(team_rank):
  print(index, "\t", team[0], "\t", team[1])
  if index > 50: break

home_field 2.1904032230377197
0 	 Ohio St 	 89.30817413330078
1 	 LSU 	 82.3166732788086
2 	 Clemson 	 80.26665496826172
3 	 Alabama 	 77.13928985595703
4 	 Wisconsin 	 74.32945251464844
5 	 Penn St 	 71.96751403808594
6 	 Oregon 	 71.77298736572266
7 	 Georgia 	 70.91683959960938
8 	 Auburn 	 70.3792495727539
9 	 Florida 	 69.5384292602539
10 	 Oklahoma 	 69.3042221069336
11 	 Notre Dame 	 68.09016418457031
12 	 Michigan 	 68.07286071777344
13 	 Utah 	 66.71409606933594
14 	 Baylor 	 64.8388442993164
15 	 Iowa 	 64.39042663574219
16 	 Texas 	 63.63643264770508
17 	 Washington 	 63.49116897583008
18 	 Minnesota 	 62.31975173950195
19 	 UCF 	 61.15861892700195
20 	 Memphis 	 60.67900085449219
21 	 Iowa St 	 60.4447135925293
22 	 Texas A&M 	 60.196258544921875
23 	 Navy 	 59.87948226928711
24 	 Kansas St 	 59.258384704589844
25 	 Oklahoma St 	 58.992679595947266
26 	 USC 	 57.404109954833984
27 	 Appalachian St 	 56.662986755371094
28 	 Cincinnati 	 56.448692321777344
29 	 TCU 	 56.09508